In [3]:
import awkward as ak
import numpy as np
from coffea import nanoevents
nanoevents.NanoAODSchema.mixins["PFCands"] = "PFCand"

import warnings
warnings.filterwarnings("ignore", message="Found duplicate branch ")

fname = "root://xcache//store/user/lpcpfnano/cmantill/v2_2/2017/HWW/GluGluHToWWToLNuQQ_M125_TuneCP5_PSweight_13TeV-powheg2-jhugen727-pythia8/GluGluHToWWToLNuQQ/211115_173633/0000/nano_mc2017_1-1.root"

# change entry_start and entry_stop here
events = nanoevents.NanoEventsFactory.from_root(fname, 
                                                schemaclass = nanoevents.NanoAODSchema, 
                                                entry_start=99, entry_stop=100).events()

In [4]:
events

<NanoEventsArray [<event 1:88:8798>] type='1 * event'>

In [5]:
good_muons = (
            (events.Muon.pt > 30)
            & (np.abs(events.Muon.eta) < 2.4)
            & (np.abs(events.Muon.dz) < 0.1)
            & (np.abs(events.Muon.dxy) < 0.05)
            & (events.Muon.sip3d <= 4.0)
            & events.Muon.mediumId
        )

good_electrons = (
            (events.Electron.pt > 38)
            & (np.abs(events.Electron.eta) < 2.4)
            & ((np.abs(events.Electron.eta) < 1.44) | (np.abs(events.Electron.eta) > 1.57))
            & (np.abs(events.Electron.dz) < 0.1)
            & (np.abs(events.Electron.dxy) < 0.05)
            & (events.Electron.sip3d <= 4.0)
            & (events.Electron.mvaFall17V2noIso_WP90)
        )

goodleptons = ak.concatenate([events.Muon[good_muons], events.Electron[good_electrons]], axis=1)    # concat muons and electrons
goodleptons = goodleptons[ak.argsort(goodleptons.pt, ascending=False)]      # sort by pt
candidatelep = ak.firsts(goodleptons) 

print(candidatelep)

[Muon]
